####IMPORTS

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#imports required
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical


####DATA PREPROCESSING

In [ ]:
filename = "shakespeare.txt"
data = open(filename, 'r', encoding='utf-8').read()
data = data.lower()

In [ ]:
maxlen = 40 #setting the maximum length of the window as 40
stride = 3
input_seq = []
target_seq= []
#creating the input and target sequences
for i in range(0, len(data) - maxlen, stride):
    input_seq.append(data[i:i+maxlen])
    target_seq.append(data[i+1:i+maxlen+1])

In [ ]:
chars = sorted(list(set(data)))
char_to_index = dict((c, i) for i, c in enumerate(chars)) #storing the indexes of the characters

# one hot vector encoding
def char_to_one_hot(c):
    one_hot_enc = np.zeros((len(chars),))
    one_hot_enc[char_to_index[c]] = 1
    return one_hot_enc

Tc = len(chars)
X = np.zeros((len(input_seq),maxlen, Tc)) #input vector
Y = np.zeros((len(target_seq),maxlen, Tc)) #output vector

for i, a in enumerate(input_seq):
    for t, char in enumerate(a):
        X[i, t, char_to_index[char]] = 1
for i, a in enumerate(target_seq):
    for t, char in enumerate(a):
        Y[i, t, char_to_index[char]] = 1

print("Number of training samples:", len(input_seq))
print("Size of vocabulary: ", len(chars))

Number of training samples: 31213
Size of vocabulary:  38


In [ ]:
X.shape

(31213, 40, 38)

In [ ]:
Y.shape

(31213, 40, 38)

####CREATING MODEL

In [ ]:
from keras.backend import dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.python.keras import regularizers
from keras.layers import Embedding

def lstm_model(input_dim):

  # Define LSTM model with three hidden layers, input layer and fully connected output layer 
  model = Sequential()
  #model.add(Embedding(input_dim=input_dim, output_dim=input_dim, input_length=maxlen))
  model.add(LSTM(128, return_sequences=True, input_shape=input_dim))
  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.1))
  model.add(LSTM(64, return_sequences=True))
  model.add(Dense(units=len(chars), activation='softmax'))

  # Compile model with Adam optimizer and cross entropy loss function
  #learning rate is set as 0.001
  learning_rate = 0.001
  optimizer = keras.optimizers.Adam(learning_rate==learning_rate)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model


####FITTING AND RUNNING THE MODEL FOR 1000 EPOCHS

In [ ]:
#fitting the model and running for 1000 epochs
model = lstm_model([maxlen,len(chars)])
model.fit(X, Y, epochs=1000, batch_size=128)

Epoch 1/1000
244/244 [==============================] - 9s 12ms/step - loss: 3.0333 - accuracy: 0.1652
Epoch 2/1000
244/244 [==============================] - 3s 13ms/step - loss: 2.7795 - accuracy: 0.2234
Epoch 3/1000
244/244 [==============================] - 3s 13ms/step - loss: 2.4214 - accuracy: 0.3084
Epoch 4/1000
244/244 [==============================] - 3s 12ms/step - loss: 2.2618 - accuracy: 0.3388
Epoch 5/1000
244/244 [==============================] - 3s 11ms/step - loss: 2.1763 - accuracy: 0.3602
Epoch 6/1000
244/244 [==============================] - 3s 12ms/step - loss: 2.1078 - accuracy: 0.3759
Epoch 7/1000
244/244 [==============================] - 3s 14ms/step - loss: 2.0495 - accuracy: 0.3927
Epoch 8/1000
244/244 [==============================] - 3s 11ms/step - loss: 1.9961 - accuracy: 0.4074
Epoch 9/1000
244/244 [==============================] - 3s 11ms/step - loss: 1.9499 - accuracy: 0.4201
Epoch 10/1000
244/244 [==============================] - 3s 11ms/step - l

####GENERATE 400 WORD SEQUENCE

In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
#seed of length 40
seed = "Thy fair maiden and her galland knights."

seed = seed.lower()
text_length = 400

# one-hot encoding the seed
input_seed = np.zeros((1, len(seed), len(chars)))
for i, char in enumerate(seed):
    input_seed[0, i, char_to_index[char]] = 1

# Generate new text character by character
def create_text(seed):
  generated_text = seed
  for i in range(text_length):
    # probability distribution for next character
    preds = model.predict(input_seed)
    # Sample from distribution and append
    next_index = np.random.choice(len(chars), p=preds[0][-1])
    next_char = index_to_char[next_index]
    generated_text += next_char
    input_seed[0, :-1, :] = input_seed[0, 1:, :]
    input_seed[0, -1, :] = 0
    input_seed[0, -1, next_index] = 1
  return generated_text

print(create_text(seed))


1/1 [==============================] - 0s 22ms/step
thy fair maiden and her galland knights.
this partial as love thee, a sad sun out to torture be takens and earthore against my self i'll forfeit,
murit him thou art not, though i view it my sone,
nor true nep invention quited
his situr's short a defence can bide?
let me provide, chilgres of my heart hads helling thiught, that is my love to make me gentle cloud,
and such absene art from thee,  
sweets with dial whereof their deaths so, '


####PERPLEXITY

In [ ]:
with open('hold-out.txt', 'r') as f:
    text = f.read()

In [ ]:
text = text.lower()
print(text)

the little love-god lying once asleep,
laid by his side his heart-inflaming brand,
whilst many nymphs that vowed chaste life to keep,
came tripping by, but in her maiden hand,
the fairest votary took up that fire,
which many legions of true hearts had warmed,
and so the general of hot desire,
was sleeping by a virgin hand disarmed.
this brand she quenched in a cool well by,
which from love's fire took heat perpetual,
growing a bath and healthful remedy,
for men discased, but i my mistress' thrall,
came there for cure and this by that i prove,  
love's fire heats water, water cools not love.



In [ ]:
eval_x = np.zeros((len(text)//40,maxlen, 38))
eval_y = np.zeros((len(text)//40,maxlen, 38))

for i in range(0,len(text)-40,40):
  if i+40+1 <= len(text):
    for t,char in enumerate(text[i:i+40]):
      eval_x[i//40,t,char_to_index[char]] = 1
      eval_y[i//40,t,char_to_index[text[i+1:i+40+1][t]]] = 1

In [ ]:
eval_x.shape

(14, 40, 38)

In [ ]:
loss,accuracy = model.evaluate(eval_x,eval_y,batch_size=128)
print(loss)
perplexity = np.exp(np.mean(loss))                

1/1 [==============================] - 1s 1s/step - loss: 3.6778 - accuracy: 0.3750
3.677837371826172


In [ ]:
#perplexity = np.exp(cross_entropy)
print('perplexity of model for the hold-out.txt is : ', perplexity)

perplexity of model for the hold-out.txt is :  39.56074630927446
